<a href="https://colab.research.google.com/github/DmeansDream/AI_Lab_Work/blob/main/AI_Lab_Work.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers -q

import os
import zipfile
import requests
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import torch
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

In [ ]:
from google.colab import drive

drive.mount('/content/drive')


In [ ]:
zip_path = "/content/drive/MyDrive/EuroSAT/EuroSAT_RGB.zip"

if os.path.exists(zip_path):
    print(f"Found dataset")
else:
    print("File not found")

In [ ]:
extract_path = "./data"

if os.path.exists(zip_path):
    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        zip_ref.extractall(extract_path)
    print("Unzipped successfully.")

    base_dir = os.path.join(extract_path, "2750")
    if not os.path.exists(base_dir):
        possible_dirs = [d for d in os.listdir(extract_path) if os.path.isdir(os.path.join(extract_path, d))]
        if possible_dirs:
            base_dir = os.path.join(extract_path, possible_dirs[0])

    print(f"Data directory at: {base_dir}")
else:
    print(f"ERROR: Could not find file")

In [ ]:
# Resize to 64x64 to be sure, and convert to tensor
transform = transforms.Compose([
    transforms.Resize((64, 64)),
    transforms.ToTensor(),
])

full_dataset = datasets.ImageFolder(root=base_dir, transform=transform)

train_size = int(0.8 * len(full_dataset))
test_size = len(full_dataset) - train_size
train_dataset, test_dataset = torch.utils.data.random_split(full_dataset, [train_size, test_size])

print(f"Total images: {len(full_dataset)}")
print(f"Training set: {len(train_dataset)}")
print(f"Test set: {len(test_dataset)}")
print(f"Classes: {full_dataset.classes}")


# Data flattening function
def extract_numpy_data(dataset):
    loader = DataLoader(dataset, batch_size=len(dataset), shuffle=False)
    data_iter = iter(loader)
    images, labels = next(data_iter)

    X = images.view(images.size(0), -1).numpy()
    y = labels.numpy()
    return X, y

print("Flattening data")
X_train, y_train = extract_numpy_data(train_dataset)
X_test, y_test = extract_numpy_data(test_dataset)

print(f"Data ready: {X_train.shape}")

First algorithm to try learning the dataset will be Naive Bayes, it is the most basic classification algorithm,

In [ ]:
# Naive Bayes

# Reducing set amount to 5000 for faster learning
subset_size = 5000
X_train_sub = X_train[:subset_size]
y_train_sub = y_train[:subset_size]

print("\n Naive Bayes Training")
start_time = time.time()

nb_model = GaussianNB()
nb_model.fit(X_train_sub, y_train_sub)

nb_time = time.time() - start_time
print(f"Training Time: {nb_time:.2f} seconds")

# Evaluation on a full set
y_pred_nb = nb_model.predict(X_test)
nb_acc = accuracy_score(y_test, y_pred_nb)
print(f"Naive Bayes Accuracy: {nb_acc*100:.2f}%")

In [ ]:
import time
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score, classification_report

# Reducing set amount to 5000 for faster learning
subset_size = 5000
X_train_sub = X_train[:subset_size]
y_train_sub = y_train[:subset_size]

# Logistic Regression
print("\n Logistic Regression Training")
start_time = time.time()

log_reg = LogisticRegression(solver='lbfgs', max_iter=500, n_jobs=-1)
log_reg.fit(X_train_sub, y_train_sub)

log_reg_time = time.time() - start_time
print(f"Training Time: {log_reg_time:.2f} seconds")

# Evaluation on a full set
y_pred_log = log_reg.predict(X_test)
log_acc = accuracy_score(y_test, y_pred_log)
print(f"Logistic Regression Accuracy: {log_acc*100:.2f}%")

In [ ]:
import time
from sklearn.ensemble import RandomForestClassifier

subset_size = 5000
X_train_sub = X_train[:subset_size]
y_train_sub = y_train[:subset_size]

print("Random Forest Training")
start_time = time.time()

rf_model = RandomForestClassifier(n_estimators=100, n_jobs=-1, random_state=393)
rf_model.fit(X_train_sub, y_train_sub)

rf_time = time.time() - start_time
print(f"Training Time: {rf_time:.2f} seconds")

# Evaluate
y_pred_rf = rf_model.predict(X_test)
rf_acc = accuracy_score(y_test, y_pred_rf)
print(f"Random Forest Accuracy: {rf_acc*100:.2f}%")

Now we set up the SVM, where we should see drastic improvement in comparison to two earlier models, since SVM can decipher not only colors but also more complex relationships between them.

In [ ]:
import time
from sklearn.svm import SVC

subset_size = 5000
X_train_sub = X_train[:subset_size]
y_train_sub = y_train[:subset_size]

print("\n SVM Training")
start_time = time.time()

svm_model = SVC(kernel='rbf', C=1.0, cache_size=1000)
svm_model.fit(X_train_sub, y_train_sub)

svm_time = time.time() - start_time
print(f"Training Time: {svm_time:.2f} seconds")

# Evaluation
y_pred_svm = svm_model.predict(X_test)
svm_acc = accuracy_score(y_test, y_pred_svm)
print(f"SVM Accuracy: {svm_acc*100:.2f}%")

Then we upload and set up data for pre trained transformer model.

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

vit_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
])

vit_dataset = datasets.ImageFolder(root=base_dir, transform=vit_transform)

train_size = int(0.8 * len(vit_dataset))
test_size = len(vit_dataset) - train_size
train_dataset_vit, test_dataset_vit = torch.utils.data.random_split(vit_dataset, [train_size, test_size])

train_loader = DataLoader(train_dataset_vit, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset_vit, batch_size=32, shuffle=False)

print("Data prepared")

from transformers import ViTForImageClassification, ViTConfig

model = ViTForImageClassification.from_pretrained(
    'google/vit-base-patch16-224-in21k',
    num_labels=10
)

model.to(device)
print("Model ready\n")

Conduct training and evaluation of transformer model.

In [ ]:
from torch.optim import AdamW
from tqdm import tqdm

optimizer = AdamW(model.parameters(), lr=5e-5)

print("ViT Training")
start_time = time.time()
model.train()

for batch in tqdm(train_loader):
    images, labels = batch
    images, labels = images.to(device), labels.to(device)

    # Forward pass
    outputs = model(images, labels=labels)
    loss = outputs.loss

    # Backward pass
    loss.backward()
    optimizer.step()
    optimizer.zero_grad()

vit_train_time = time.time() - start_time
print(f"Training Time: {vit_train_time:.2f} seconds")

print("Evaluating")
model.eval()
correct = 0
total = 0

with torch.no_grad():
    for batch in tqdm(test_loader):
        images, labels = batch
        images, labels = images.to(device), labels.to(device)

        outputs = model(images)
        predictions = torch.argmax(outputs.logits, dim=-1)

        correct += (predictions == labels).sum().item()
        total += labels.size(0)

vit_acc = correct / total
print(f"\n Transformer Accuracy: {vit_acc*100:.2f}%")